# DQN Training for Fire-Rescue Environment

#### Deep Q-Network - Value-Based Reinforcement Learning
 
This notebook includes:
 - 10+ hyperparameter configurations
 - Training with different settings
 - Logging and visualization
 - Model saving and evaluation

In [ ]:
!pip install gymnasium stable-baselines3 tensorboard matplotlib pandas seaborn tqdm

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# CELL 2: Import Libraries
# ═══════════════════════════════════════════════════════════════════════
import sys
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Stable Baselines
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback, CallbackList
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Add project root to path
project_root = Path.cwd().parent if 'training' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(project_root))

# Import custom environment
from environment.custom_env import FireRescueEnv

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)

print("✓ All libraries imported successfully")
print(f"✓ Project root: {project_root}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# CELL 3: Setup Directories
# ═══════════════════════════════════════════════════════════════════════
"""
Create necessary directories for models and logs
"""
# Create directory structure
base_dir = project_root / "models" / "dqn"
logs_dir = base_dir / "training_logs"
tensorboard_dir = base_dir / "tensorboard"
results_dir = project_root / "results"
plots_dir = results_dir / "plots"

for directory in [base_dir, logs_dir, tensorboard_dir, results_dir, plots_dir]:
    directory.mkdir(parents=True, exist_ok=True)

print("✓ Directories created:")
print(f"  - Models: {base_dir}")
print(f"  - Logs: {logs_dir}")
print(f"  - Tensorboard: {tensorboard_dir}")
print(f"  - Results: {results_dir}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# cell 4: DQN HYPERPARAMETER CONFIGURATIONS
# ═══════════════════════════════════════════════════════════════════════
"""
Focused on key hyperparameters mentioned by instructor:
- learning_rate (lr)
- gamma (discount factor)
- batch_size
- epsilon_start (exploration_initial_eps)
- epsilon_end (exploration_final_eps)
- epsilon_decay (exploration_fraction)

Standardized across all configs:
- buffer_size = 50000 (same for all)
- learning_starts = 1000 (same for all)
- tau = 1.0 (hard target network updates)

Network architecture (policy_kwargs):
- Most use [64, 64] (standard)
- Some use [128, 128] (deeper) or [32, 32] (shallower)
"""

DQN_CONFIGS = {
    # ─────────────────────────────────────────────────────────────────────
    # Config 1: BASELINE (Good starting point)
    # ─────────────────────────────────────────────────────────────────────
    "config_1_baseline": {
        "learning_rate": 1e-3,
        "gamma": 0.99,
        "batch_size": 32,
        "exploration_initial_eps": 1.0,
        "exploration_final_eps": 0.05,
        "exploration_fraction": 0.2,
        # Standardized parameters
        "buffer_size": 50000,
        "learning_starts": 1000,
        "tau": 1.0,
        "policy_kwargs": dict(net_arch=[64, 64]),
        "description": "Baseline - Standard settings"
    },
    
    # ─────────────────────────────────────────────────────────────────────
    # Config 2: HIGH LEARNING RATE (Too aggressive - may not converge)
    # ─────────────────────────────────────────────────────────────────────
    "config_2_high_lr": {
        "learning_rate": 5e-3,  # Very high
        "gamma": 0.99,
        "batch_size": 32,
        "exploration_initial_eps": 1.0,
        "exploration_final_eps": 0.05,
        "exploration_fraction": 0.2,
        # Standardized
        "buffer_size": 50000,
        "learning_starts": 1000,
        "tau": 1.0,
        "policy_kwargs": dict(net_arch=[64, 64]),
        "description": "High LR - Unstable learning"
    },
    
    # ─────────────────────────────────────────────────────────────────────
    # Config 3: LOW LEARNING RATE (Too slow - won't learn much)
    # ─────────────────────────────────────────────────────────────────────
    "config_3_low_lr": {
        "learning_rate": 1e-5,  # Very low
        "gamma": 0.99,
        "batch_size": 32,
        "exploration_initial_eps": 1.0,
        "exploration_final_eps": 0.05,
        "exploration_fraction": 0.2,
        # Standardized
        "buffer_size": 50000,
        "learning_starts": 1000,
        "tau": 1.0,
        "policy_kwargs": dict(net_arch=[64, 64]),
        "description": "Low LR - Slow learning"
    },
    
    # ─────────────────────────────────────────────────────────────────────
    # Config 4: LARGE BATCH (More stable but slower)
    # ─────────────────────────────────────────────────────────────────────
    "config_4_large_batch": {
        "learning_rate": 1e-3,
        "gamma": 0.99,
        "batch_size": 128,  # Large batch
        "exploration_initial_eps": 1.0,
        "exploration_final_eps": 0.05,
        "exploration_fraction": 0.2,
        # Standardized
        "buffer_size": 50000,
        "learning_starts": 1000,
        "tau": 1.0,
        "policy_kwargs": dict(net_arch=[64, 64]),
        "description": "Large Batch - Stable gradients"
    },
    
    # ─────────────────────────────────────────────────────────────────────
    # Config 5: SMALL BATCH (Noisier but faster)
    # ─────────────────────────────────────────────────────────────────────
    "config_5_small_batch": {
        "learning_rate": 1e-3,
        "gamma": 0.99,
        "batch_size": 16,  # Small batch
        "exploration_initial_eps": 1.0,
        "exploration_final_eps": 0.05,
        "exploration_fraction": 0.2,
        # Standardized
        "buffer_size": 50000,
        "learning_starts": 1000,
        "tau": 1.0,
        "policy_kwargs": dict(net_arch=[64, 64]),
        "description": "Small Batch - Fast updates"
    },
    
    # ─────────────────────────────────────────────────────────────────────
    # Config 6: LOW GAMMA (Short-sighted - doesn't plan ahead)
    # ─────────────────────────────────────────────────────────────────────
    "config_6_low_gamma": {
        "learning_rate": 1e-3,
        "gamma": 0.95,  # Low discount factor
        "batch_size": 32,
        "exploration_initial_eps": 1.0,
        "exploration_final_eps": 0.05,
        "exploration_fraction": 0.2,
        # Standardized
        "buffer_size": 50000,
        "learning_starts": 1000,
        "tau": 1.0,
        "policy_kwargs": dict(net_arch=[64, 64]),
        "description": "Low Gamma - Myopic policy"
    },
    
    # ─────────────────────────────────────────────────────────────────────
    # Config 7: HIGH GAMMA (Very forward-looking)
    # ─────────────────────────────────────────────────────────────────────
    "config_7_high_gamma": {
        "learning_rate": 1e-3,
        "gamma": 0.995,  # High discount factor
        "batch_size": 32,
        "exploration_initial_eps": 1.0,
        "exploration_final_eps": 0.05,
        "exploration_fraction": 0.2,
        # Standardized
        "buffer_size": 50000,
        "learning_starts": 1000,
        "tau": 1.0,
        "policy_kwargs": dict(net_arch=[64, 64]),
        "description": "High Gamma - Long-term planning"
    },
    
    # ─────────────────────────────────────────────────────────────────────
    # Config 8: FAST EXPLORATION DECAY (Stops exploring too soon)
    # ─────────────────────────────────────────────────────────────────────
    "config_8_fast_decay": {
        "learning_rate": 1e-3,
        "gamma": 0.99,
        "batch_size": 32,
        "exploration_initial_eps": 1.0,
        "exploration_final_eps": 0.1,  # Higher final epsilon
        "exploration_fraction": 0.1,  # Fast decay
        # Standardized
        "buffer_size": 50000,
        "learning_starts": 1000,
        "tau": 1.0,
        "policy_kwargs": dict(net_arch=[64, 64]),
        "description": "Fast Decay - Premature exploitation"
    },
    
    # ─────────────────────────────────────────────────────────────────────
    # Config 9: SLOW EXPLORATION DECAY (Explores longer)
    # ─────────────────────────────────────────────────────────────────────
    "config_9_slow_decay": {
        "learning_rate": 1e-3,
        "gamma": 0.99,
        "batch_size": 32,
        "exploration_initial_eps": 1.0,
        "exploration_final_eps": 0.01,  # Very low final epsilon
        "exploration_fraction": 0.4,  # Slow decay
        # Standardized
        "buffer_size": 50000,
        "learning_starts": 1000,
        "tau": 1.0,
        "policy_kwargs": dict(net_arch=[64, 64]),
        "description": "Slow Decay - Extended exploration"
    },
    
    # ─────────────────────────────────────────────────────────────────────
    # Config 10: OPTIMIZED (Best guess based on task)
    # ─────────────────────────────────────────────────────────────────────
    "config_10_optimized": {
        "learning_rate": 3e-4,  # Sweet spot
        "gamma": 0.99,
        "batch_size": 64,  # Good balance
        "exploration_initial_eps": 1.0,
        "exploration_final_eps": 0.02,
        "exploration_fraction": 0.3,  # Balanced exploration
        # Standardized
        "buffer_size": 50000,
        "learning_starts": 1000,
        "tau": 1.0,
        "policy_kwargs": dict(net_arch=[128, 128]),  # Deeper network
        "description": "Optimized - Well-balanced settings"
    },
    
     # ─────────────────────────────────────────────────────────────────────
    # Config 11: Zero epsilon start and end
    # ─────────────────────────────────────────────────────────────────────
    "config_11_low_gamma": {
        "learning_rate": 1e-3,
        "gamma": 0.95,
        "batch_size": 32,
        "exploration_initial_eps": 0,
        "exploration_final_eps": 0,
        "exploration_fraction": 0.2,
        # Standardized
        "buffer_size": 50000,
        "learning_starts": 1000,
        "tau": 1.0,
        "policy_kwargs": dict(net_arch=[64, 64]),
        "description": "Zero Epsilon - No exploration"
    }
}

# ═══════════════════════════════════════════════════════════════════════
# CONFIGURATION SUMMARY TABLE
# ═══════════════════════════════════════════════════════════════════════
print(f"✓ Defined {len(DQN_CONFIGS)} DQN configurations\n")
print("=" * 100)
print(f"{'Config':<25} {'LR':<10} {'Gamma':<8} {'Batch':<8} {'ε_start':<10} {'ε_end':<10} {'ε_decay':<10} {'Network':<15}")
print("=" * 100)

for config_name, config in DQN_CONFIGS.items():
    net_arch = str(config['policy_kwargs']['net_arch'])
    print(f"{config_name:<25} {config['learning_rate']:<10.0e} {config['gamma']:<8.3f} "
          f"{config['batch_size']:<8} {config['exploration_initial_eps']:<10.2f} "
          f"{config['exploration_final_eps']:<10.2f} {config['exploration_fraction']:<10.2f} {net_arch:<15}")

print("=" * 100)
print("\nKey Insights:")
print("  • Configs 2 & 3: Test extreme learning rates")
print("  • Configs 4 & 5: Test batch size impact")
print("  • Configs 6 & 7: Test discount factor (short vs long-term)")
print("  • Configs 8 & 9: Test exploration strategy")
print("  • Config 10: Optimized combination")
print("\nAll configs use:")
print("  • buffer_size = 50,000 (standardized)")
print("  • learning_starts = 1,000 (standardized)")
print("  • tau = 1.0 (hard target updates)")
print("=" * 100)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# CELL 5: Training Configuration
# ═══════════════════════════════════════════════════════════════════════
"""
Set training parameters
"""
# Training settings
TOTAL_TIMESTEPS = 150000  # Total training steps per configuration
EVAL_FREQ = 5000          # Evaluate every N steps
N_EVAL_EPISODES = 10      # Number of episodes for evaluation
SAVE_FREQ = 10000         # Save model checkpoint every N steps

print("Training Configuration:")
print(f"  - Total Timesteps: {TOTAL_TIMESTEPS:,}")
print(f"  - Evaluation Frequency: {EVAL_FREQ:,}")
print(f"  - Evaluation Episodes: {N_EVAL_EPISODES}")
print(f"  - Checkpoint Frequency: {SAVE_FREQ:,}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# CELL 6: Training Function
# ═══════════════════════════════════════════════════════════════════════
"""
Function to train a single DQN configuration
"""

def train_dqn_config(config_name, config, total_timesteps=TOTAL_TIMESTEPS):
    """
    Train DQN with given configuration
    
    Args:
        config_name: Name of the configuration
        config: Dictionary of hyperparameters
        total_timesteps: Total training steps
    
    Returns:
        Dictionary with training results
    """
    print(f"\n{'='*70}")
    print(f"Training: {config_name}")
    print(f"Description: {config['description']}")
    print(f"{'='*70}")
    
    # Create environment
    env = FireRescueEnv(grid_size=10, max_time=180)
    env = Monitor(env)
    
    # Create eval environment
    eval_env = FireRescueEnv(grid_size=10, max_time=180)
    eval_env = Monitor(eval_env)
    
    # Create model directory for this config
    model_dir = base_dir / config_name
    model_dir.mkdir(exist_ok=True)
    
    # Setup callbacks
    eval_callback = EvalCallback(
        eval_env,
        best_model_save_path=str(model_dir),
        log_path=str(logs_dir / config_name),
        eval_freq=EVAL_FREQ,
        n_eval_episodes=N_EVAL_EPISODES,
        deterministic=True,
        render=False,
        verbose=1
    )
    
    checkpoint_callback = CheckpointCallback(
        save_freq=SAVE_FREQ,
        save_path=str(model_dir / "checkpoints"),
        name_prefix="dqn_checkpoint",
        save_replay_buffer=True,
        save_vecnormalize=True
    )
    
    callback_list = CallbackList([eval_callback, checkpoint_callback])
    
    # Extract hyperparameters (remove description)
    train_config = {k: v for k, v in config.items() if k != 'description'}
    
    # Create DQN model
    model = DQN(
        "MlpPolicy",
        env,
        verbose=1,
        seed=SEED,
        tensorboard_log=str(tensorboard_dir / config_name),
        **train_config
    )
    
    # Train
    start_time = datetime.now()
    print(f"\nStarting training at {start_time.strftime('%H:%M:%S')}")
    
    try:
        model.learn(
            total_timesteps=total_timesteps,
            callback=callback_list,
            progress_bar=True
        )
        
        training_time = (datetime.now() - start_time).total_seconds()
        print(f"\n✓ Training completed in {training_time:.1f} seconds")
        
        # Save final model
        final_model_path = model_dir / "final_model"
        model.save(str(final_model_path))
        print(f"✓ Model saved to {final_model_path}")
        
        # Evaluate final model
        print("\nEvaluating final model...")
        mean_reward, std_reward = evaluate_policy(
            model, 
            eval_env, 
            n_eval_episodes=20,
            deterministic=True
        )
        
        print(f"✓ Mean Reward: {mean_reward:.2f} ± {std_reward:.2f}")
        
        # Calculate success rate
        success_count = 0
        episode_rewards = []
        episode_lengths = []
        
        for _ in range(20):
            obs, info = eval_env.reset()
            episode_reward = 0
            episode_length = 0
            done = False
            
            while not done:
                action, _states = model.predict(obs, deterministic=True)
                obs, reward, terminated, truncated, info = eval_env.step(action)
                done = terminated or truncated
                episode_reward += reward
                episode_length += 1
            
            episode_rewards.append(episode_reward)
            episode_lengths.append(episode_length)
            
            if info.get('success', False):
                success_count += 1
        
        success_rate = success_count / 20
        
        # Store results
        results = {
            'config_name': config_name,
            'description': config['description'],
            'mean_reward': mean_reward,
            'std_reward': std_reward,
            'success_rate': success_rate,
            'mean_episode_length': np.mean(episode_lengths),
            'std_episode_length': np.std(episode_lengths),
            'training_time': training_time,
            'total_timesteps': total_timesteps,
            'hyperparameters': train_config
        }
        
        # Save results to JSON
        results_file = model_dir / "results.json"
        with open(results_file, 'w') as f:
            # Convert numpy types to Python types for JSON
            json_results = {k: (float(v) if isinstance(v, (np.floating, np.integer)) else v) 
                          for k, v in results.items() if k != 'hyperparameters'}
            json_results['hyperparameters'] = {
                k: (float(v) if isinstance(v, (np.floating, np.integer)) else v)
                for k, v in results['hyperparameters'].items() if k != 'policy_kwargs'
            }
            json.dump(json_results, f, indent=4)
        
        print(f"✓ Results saved to {results_file}")
        
        # Cleanup
        env.close()
        eval_env.close()
        
        return results
        
    except Exception as e:
        print(f"\n✗ Training failed: {e}")
        env.close()
        eval_env.close()
        return None

print("✓ Training function defined")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# CELL 7: Train All Configurations
# ═══════════════════════════════════════════════════════════════════════
"""
Train all DQN configurations
WARNING: This will take several hours!
"""

# Store all results
all_results = []

print(f"\n{'#'*70}")
print(f"STARTING FULL TRAINING RUN")
print(f"Training {len(DQN_CONFIGS)} configurations")
print(f"Estimated time: {len(DQN_CONFIGS) * 15} - {len(DQN_CONFIGS) * 30} minutes")
print(f"{'#'*70}\n")

# Train each configuration
for config_name, config in DQN_CONFIGS.items():
    results = train_dqn_config(config_name, config)
    
    if results:
        all_results.append(results)
        print(f"\n✓ Completed {len(all_results)}/{len(DQN_CONFIGS)} configurations")
    else:
        print(f"\n✗ Failed configuration: {config_name}")
    
    print(f"\n{'='*70}\n")

print(f"\n{'#'*70}")
print(f"TRAINING COMPLETE!")
print(f"Successfully trained: {len(all_results)}/{len(DQN_CONFIGS)} configurations")
print(f"{'#'*70}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# CELL 8: Create Results DataFrame
# ═══════════════════════════════════════════════════════════════════════
"""
Organize results into a pandas DataFrame for analysis
"""

# Check if we have any results
if len(all_results) == 0:
    print("\n" + "="*70)
    print("NO SUCCESSFUL TRAINING RUNS")
    print("="*70)
    print("\nPossible issues:")
    print("1. Missing packages: Run 'pip install stable-baselines3[extra]'")
    print("2. Environment import error: Check environment/custom_env.py exists")
    print("3. GPU/memory issues: Try reducing TOTAL_TIMESTEPS in Cell 5")
    print("\nPlease fix the errors above and re-run Cell 7")
    print("="*70)
else:
    # Create DataFrame
    results_df = pd.DataFrame(all_results)

    # Display results
    print("\n" + "="*70)
    print("DQN TRAINING RESULTS SUMMARY")
    print("="*70)
    print(results_df[['config_name', 'mean_reward', 'std_reward', 'success_rate', 'training_time']].to_string(index=False))

    # Save to CSV
    csv_path = results_dir / "dqn_results.csv"
    results_df.to_csv(csv_path, index=False)
    print(f"\n✓ Results saved to {csv_path}")

    # Find best configuration
    best_config = results_df.loc[results_df['mean_reward'].idxmax()]
    print(f"\n{'='*70}")
    print("BEST CONFIGURATION")
    print(f"{'='*70}")
    print(f"Name: {best_config['config_name']}")
    print(f"Description: {best_config['description']}")
    print(f"Mean Reward: {best_config['mean_reward']:.2f} ± {best_config['std_reward']:.2f}")
    print(f"Success Rate: {best_config['success_rate']*100:.1f}%")
    print(f"Training Time: {best_config['training_time']:.1f}s")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# CELL 9: Visualize Results - Comparison Plot
# ═══════════════════════════════════════════════════════════════════════
"""
Create comprehensive visualization of results
"""

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 12)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Sort by mean reward
results_sorted = results_df.sort_values('mean_reward', ascending=False)

# 1. Mean Reward Comparison
ax1 = axes[0, 0]
bars = ax1.barh(range(len(results_sorted)), results_sorted['mean_reward'], 
                xerr=results_sorted['std_reward'], capsize=5)
ax1.set_yticks(range(len(results_sorted)))
ax1.set_yticklabels(results_sorted['config_name'], fontsize=9)
ax1.set_xlabel('Mean Reward', fontsize=12, fontweight='bold')
ax1.set_title('DQN Configuration Performance Comparison', fontsize=14, fontweight='bold')
ax1.axvline(x=0, color='red', linestyle='--', alpha=0.5)

# Color bars by performance
colors = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(bars)))
for bar, color in zip(bars, colors):
    bar.set_color(color)

ax1.grid(axis='x', alpha=0.3)

# 2. Success Rate
ax2 = axes[0, 1]
success_sorted = results_df.sort_values('success_rate', ascending=False)
bars2 = ax2.barh(range(len(success_sorted)), success_sorted['success_rate'] * 100)
ax2.set_yticks(range(len(success_sorted)))
ax2.set_yticklabels(success_sorted['config_name'], fontsize=9)
ax2.set_xlabel('Success Rate (%)', fontsize=12, fontweight='bold')
ax2.set_title('Mission Success Rate by Configuration', fontsize=14, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

# Color bars
colors2 = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(bars2)))
for bar, color in zip(bars2, colors2):
    bar.set_color(color)

# 3. Training Time vs Performance
ax3 = axes[1, 0]
scatter = ax3.scatter(results_df['training_time'], results_df['mean_reward'], 
                     s=results_df['success_rate']*500, alpha=0.6, 
                     c=results_df['mean_reward'], cmap='RdYlGn')
ax3.set_xlabel('Training Time (seconds)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Mean Reward', fontsize=12, fontweight='bold')
ax3.set_title('Training Efficiency: Time vs Performance', fontsize=14, fontweight='bold')
ax3.grid(alpha=0.3)

# Add labels for best and worst
best_idx = results_df['mean_reward'].idxmax()
worst_idx = results_df['mean_reward'].idxmin()
ax3.annotate('Best', xy=(results_df.loc[best_idx, 'training_time'], 
                         results_df.loc[best_idx, 'mean_reward']),
            xytext=(10, 10), textcoords='offset points',
            bbox=dict(boxstyle='round,pad=0.5', fc='green', alpha=0.3),
            arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

# Colorbar
plt.colorbar(scatter, ax=ax3, label='Mean Reward')

# 4. Hyperparameter Impact - Learning Rate
ax4 = axes[1, 1]
lr_values = []
rewards = []
for _, row in results_df.iterrows():
    if 'hyperparameters' in row and row['hyperparameters']:
        lr = row['hyperparameters'].get('learning_rate', None)
        if lr:
            lr_values.append(lr)
            rewards.append(row['mean_reward'])

if lr_values:
    ax4.scatter(lr_values, rewards, s=100, alpha=0.6, c=rewards, cmap='RdYlGn')
    ax4.set_xlabel('Learning Rate', fontsize=12, fontweight='bold')
    ax4.set_ylabel('Mean Reward', fontsize=12, fontweight='bold')
    ax4.set_title('Impact of Learning Rate on Performance', fontsize=14, fontweight='bold')
    ax4.set_xscale('log')
    ax4.grid(alpha=0.3)

plt.tight_layout()
plot_path = plots_dir / "dqn_comparison.png"
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
print(f"\n✓ Comparison plot saved to {plot_path}")
plt.show()

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# CELL 10: Detailed Analysis - Top 3 Configurations
# ═══════════════════════════════════════════════════════════════════════
"""
Detailed analysis of top 3 performing configurations
"""

print("\n" + "="*70)
print("TOP 3 CONFIGURATIONS - DETAILED ANALYSIS")
print("="*70)

top3 = results_df.nlargest(3, 'mean_reward')

for idx, (_, config) in enumerate(top3.iterrows(), 1):
    print(f"\n{'─'*70}")
    print(f"#{idx}: {config['config_name']}")
    print(f"{'─'*70}")
    print(f"Description: {config['description']}")
    print(f"Mean Reward: {config['mean_reward']:.2f} ± {config['std_reward']:.2f}")
    print(f"Success Rate: {config['success_rate']*100:.1f}%")
    print(f"Avg Episode Length: {config['mean_episode_length']:.1f} ± {config['std_episode_length']:.1f} steps")
    print(f"Training Time: {config['training_time']:.1f} seconds")
    
    if 'hyperparameters' in config and config['hyperparameters']:
        print(f"\nKey Hyperparameters:")
        hparams = config['hyperparameters']
        print(f"  - Learning Rate: {hparams.get('learning_rate', 'N/A')}")
        print(f"  - Batch Size: {hparams.get('batch_size', 'N/A')}")
        print(f"  - Buffer Size: {hparams.get('buffer_size', 'N/A')}")
        print(f"  - Gamma: {hparams.get('gamma', 'N/A')}")
        print(f"  - Exploration Fraction: {hparams.get('exploration_fraction', 'N/A')}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# CELL 11: Export Summary for Report
# ═══════════════════════════════════════════════════════════════════════
"""
Create summary tables and statistics for the final report
"""

# Summary statistics
summary = {
    'total_configs': len(all_results),
    'best_config': best_config['config_name'],
    'best_mean_reward': float(best_config['mean_reward']),
    'best_success_rate': float(best_config['success_rate']),
    'avg_mean_reward': float(results_df['mean_reward'].mean()),
    'std_mean_reward': float(results_df['mean_reward'].std()),
    'avg_success_rate': float(results_df['success_rate'].mean()),
    'total_training_time': float(results_df['training_time'].sum()),
}

# Save summary
summary_path = results_dir / "dqn_summary.json"
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=4)

print("\n" + "="*70)
print("DQN TRAINING SUMMARY")
print("="*70)
for key, value in summary.items():
    print(f"{key.replace('_', ' ').title()}: {value}")

print(f"\n✓ Summary saved to {summary_path}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# CELL 12: Instructions for Next Steps
# ═══════════════════════════════════════════════════════════════════════
"""
Next steps after DQN training
"""

print("\n" + "="*70)
print("NEXT STEPS")
print("="*70)
print("\n1. ✓ DQN Training Complete!")
print(f"   - Trained {len(all_results)} configurations")
print(f"   - Best model: {best_config['config_name']}")
print(f"   - Success rate: {best_config['success_rate']*100:.1f}%")

print("\n2. → Train Other Algorithms:")
print("   - PPO (Policy Gradient)")
print("   - A2C (Actor-Critic)")
print("   - REINFORCE")

print("\n3. → Compare All Algorithms:")
print("   - Create comparison plots")
print("   - Analyze which works best")
print("   - Write discussion section")

print("\n4. → Create Demonstration:")
print("   - Record video with best model")
print("   - Show Unity visualization")
print("   - Explain agent behavior")

print("\n5. → Files Generated:")
print(f"   - Models: {base_dir}")
print(f"   - Results: {csv_path}")
print(f"   - Plots: {plot_path}")
print(f"   - Summary: {summary_path}")

print("\n" + "="*70)
print("Ready to train PPO next? 🚀")
print("="*70)